In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
def run_ex1_43_full(initial_lr, iterations = 600, lambda_reg = 0.001, beta1 = 0.9, beta2 = 0.999, epsilon = 1e-8, master_seed = 42, print_every = 50, verbose = True):
    np.random.seed(master_seed)
    n_samples = 500
    area = np.random.uniform(10, 50, n_samples)
    expected_bedrooms = np.clip((area/15), 0, 4)
    bedrooms = np.random.normal(expected_bedrooms, 0.5)
    bedrooms = np.round(bedrooms).astype(int)
    bedrooms = np.clip(bedrooms, 0, 4)
    age = np.random.uniform(0, 30, n_samples)
    noise = np.random.normal(0, 2, n_samples)
    rent = 1.5 * area + 2 * bedrooms - 0.8 * age + 5 + noise

    X = np.column_stack((area, bedrooms, age))
    y = rent

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
    scaler = StandardScaler()
    X_train_standardized = scaler.fit_transform(X_train)
    X_test_standardized = scaler.transform(X_test)

    mean = scaler.mean_
    scale = scaler.scale_
    print('mean = ', np.round(mean, 4))
    print('scale = ', np.round(scale, 4))

    w = np.random.rand(X_train_standardized.shape[1])
    b = np.random.rand()

    eta_min = 1e-6
    T_0 = 2000
    T_i = T_0
    T_mult = 2
    T_cur = 0
    cycle = 0
    gamma = 0.98
 
    m_w = np.zeros_like(w)
    m_b = 0
    v_w = np.zeros_like(w)
    v_b = 0

    lr_history = []
    loss_history_train = []
    loss_history_test = []
    w_history = []
    b_history = []

    for i in range(iterations):
        idx = np.random.randint(0, len(X_train_standardized))
        x_i = X_train_standardized[idx]
        y_i = y_train[idx]

        y_pred_i = np.dot(x_i, w) + b
        error_i = y_i - y_pred_i

        w_gradient = (-2) * x_i * error_i + 2 * lambda_reg * w
        b_gradient = (-2) * error_i

        m_w = beta1 * m_w + (1 - beta1) * w_gradient
        m_b = beta1 * m_b + (1 - beta1) * b_gradient
        v_w = beta2 * v_w + (1 - beta2) * (w_gradient ** 2)
        v_b = beta2 * v_b + (1 - beta2) * (b_gradient ** 2)

        m_w_hat = m_w / (1 - beta1 ** (i+1))
        m_b_hat = m_b / (1 - beta1 ** (i+1))
        v_w_hat = v_w / (1 - beta2 ** (i+1))
        v_b_hat = v_b / (1 - beta2 ** (i+1))

        cos_inner = min(1, T_cur/ T_i)
        learning_rate = eta_min + (1/2) * (initial_lr - eta_min) * (1 + math.cos(cos_inner * math.pi))
        lr_history.append(learning_rate)

        w -= learning_rate * (m_w_hat / (np.sqrt(v_w_hat) + epsilon))
        b -= learning_rate * (m_b_hat / (np.sqrt(v_b_hat) + epsilon))

        w_history.append(w.copy())
        b_history.append(b)

        T_cur +=1
        if T_cur >= T_i:
            T_cur = 0
            cycle += 1
            T_i = int(T_i * T_mult)
            initial_lr *= gamma
        
        y_pred_train = np.dot(X_train_standardized, w) + b
        error_train = y_train - y_pred_train
        loss_train = np.mean(error_train ** 2) + lambda_reg * np.sum(w ** 2)
        loss_history_train.append(loss_train)

        y_pred_test = np.dot(X_test_standardized, w) + b
        error_test = y_test - y_pred_test
        loss_test = np.mean(error_test ** 2)
        loss_history_test.append(loss_test)

        if verbose and (i % print_every == 0 or i == (iterations-1)):
            print(f'Iteration {i} : w = {np.round(w, 4)}, b = {b:.4f}, Train Loss = {loss_train:.4f}, Test Loss = {loss_test:.4f}')
    
    loss_history_test_arr = np.array(loss_history_test)
    min_index = np.argmin(loss_history_test_arr)
    min_test_loss = loss_history_test_arr[min_index]
    best_w = w_history[min_index]
    best_b = b_history[min_index]
    lr_at_best = lr_history[min_index]

    df_summary = pd.DataFrame({
        'Final Train Loss' : [np.round(loss_history_train[iterations-1], 4)],
        'Final Test Loss' : [np.round(loss_history_test[iterations-1], 4)],
        'Best Iteration' : [min_index],
        'Train Loss @ Best Test' : [np.round(loss_history_train[min_index], 4)],
        'Best Test Loss' : [np.round(min_test_loss, 4)],
        'w (params)' : [np.round(best_w, 4)],
        'b (bias)' : [np.round(best_b, 4)]
    })

    return min_test_loss, min_index, best_w, best_b, df_summary, loss_history_train, loss_history_test, lr_history


min_test_loss, min_index, best_w, best_b, df_summary, loss_history_train, loss_history_test, lr_history = run_ex1_43_full(initial_lr = 0.78, iterations = 600, master_seed = 42, verbose = True)
display(df_summary)

def lr_grid_search_ex1_43(lr_start = 0.2, lr_end = 8, lr_step = 0.01, iterations = 600, master_seed = 42):
    results = []
    best_lr = None
    best_loss = np.inf

    lr_list = np.arange(lr_start, lr_end + lr_step, lr_step)

    for lr in lr_list:
        print('Testing LR = ', np.round(lr, 4))

        min_test_loss, min_index, best_w, best_b, df_summary, loss_history_train, loss_history_test, lr_history = run_ex1_43_full(initial_lr = lr, iterations = iterations, master_seed = master_seed, verbose = False)
        results.append({'lr' : lr, 'best_test_loss' : np.round(min_test_loss, 4)})

        if min_test_loss < best_loss:
            best_loss = min_test_loss
            best_lr = lr
    
    df = pd.DataFrame(results).sort_values('best_test_loss').reset_index(drop = True)
    return best_lr, df




mean =  [30.1526  1.99   14.6872]
scale =  [11.9856  1.0246  8.5246]
Iteration 0 : w = [0.92   1.0568 1.7515], b = 1.1113, Train Loss = 2113.8435, Test Loss = 1763.1461
Iteration 50 : w = [11.7859  9.0875 -5.9366], b = 31.5468, Train Loss = 142.4138, Test Loss = 129.1171
Iteration 100 : w = [13.0211  7.3352 -7.2353], b = 41.8564, Train Loss = 13.0410, Test Loss = 14.6892
Iteration 150 : w = [13.6721  5.7641 -6.3989], b = 42.8370, Train Loss = 9.3294, Test Loss = 12.4712
Iteration 200 : w = [14.4289  4.8423 -5.8439], b = 42.4303, Train Loss = 8.3472, Test Loss = 11.2872
Iteration 250 : w = [16.2054  3.4643 -7.2193], b = 42.3849, Train Loss = 4.9464, Test Loss = 6.8932
Iteration 300 : w = [17.3247  3.0207 -6.7541], b = 42.6754, Train Loss = 4.0820, Test Loss = 5.8075
Iteration 350 : w = [17.5449  3.4938 -6.4542], b = 43.1633, Train Loss = 5.3227, Test Loss = 7.1341
Iteration 400 : w = [17.6634  2.7409 -6.8835], b = 43.6281, Train Loss = 4.9642, Test Loss = 6.8148
Iteration 450 : w = [17.

,Final Train Loss,Final Test Loss,Best Iteration,Train Loss @ Best Test,Best Test Loss,w (params),b (bias)
0,4.1757,6.2106,466,3.9703,5.4947,"[18.0935, 2.2251, -6.9673]",42.5756


In [3]:
best_lr, df_lr = lr_grid_search_ex1_43(lr_start = 0.2, lr_end = 8, lr_step = 0.01, iterations = 600, master_seed = 42)
print('Grid Search 最佳的 LR = ', np.round(best_lr, 4))
display(df_lr.head(10))

Testing LR =  0.2
mean =  [30.1526  1.99   14.6872]
scale =  [11.9856  1.0246  8.5246]
Testing LR =  0.21
mean =  [30.1526  1.99   14.6872]
scale =  [11.9856  1.0246  8.5246]
Testing LR =  0.22
mean =  [30.1526  1.99   14.6872]
scale =  [11.9856  1.0246  8.5246]
Testing LR =  0.23
mean =  [30.1526  1.99   14.6872]
scale =  [11.9856  1.0246  8.5246]
Testing LR =  0.24
mean =  [30.1526  1.99   14.6872]
scale =  [11.9856  1.0246  8.5246]
Testing LR =  0.25
mean =  [30.1526  1.99   14.6872]
scale =  [11.9856  1.0246  8.5246]
Testing LR =  0.26
mean =  [30.1526  1.99   14.6872]
scale =  [11.9856  1.0246  8.5246]
Testing LR =  0.27
mean =  [30.1526  1.99   14.6872]
scale =  [11.9856  1.0246  8.5246]
Testing LR =  0.28
mean =  [30.1526  1.99   14.6872]
scale =  [11.9856  1.0246  8.5246]
Testing LR =  0.29
mean =  [30.1526  1.99   14.6872]
scale =  [11.9856  1.0246  8.5246]
Testing LR =  0.3
mean =  [30.1526  1.99   14.6872]
scale =  [11.9856  1.0246  8.5246]
Testing LR =  0.31
mean =  [30.152

,lr,best_test_loss
0,0.78,5.4947
1,0.79,5.4949
2,0.77,5.4951
3,0.80,5.4955
4,0.76,5.4956
5,0.81,5.4963
6,0.75,5.4965
7,0.82,5.4970
8,0.74,5.4979
9,0.83,5.4982
